<a href="https://colab.research.google.com/github/Machine-Learning-Tokyo/Kaggle/blob/master/Earthquake_Prediction/data_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code to download the data

In [0]:
!pip install -U -q kaggle
!mkdir -p .kaggle
!mkdir -p ~/.kaggle

In [0]:
import json
token = {"username":"your name","key":"your key"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
%%capture
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c LANL-Earthquake-Prediction -p /content

In [12]:
!ls /content/

sample_data  sample_submission.csv  test  train.csv


In [0]:
!unzip train.csv.zip && rm train.csv.zip
!unzip test.zip -d test && rm test.zip

## Code after downloading the data

In [0]:
import csv
import numpy as np

In [0]:
train_csv_path = 'train.csv'
batch_size = 32

In [0]:
class CSVReader:
  def __init__(self, csv_path, batch_size):
    csvfile = open(csv_path, "r")
    self.datareader = csv.reader(csvfile)
    next(self.datareader)
    
  def __iter__(self):
    return self
  
  def __next__(self):
    vs, ts = np.zeros((batch_size,)), np.zeros((batch_size,))
    for i in range(batch_size):
      vs[i], ts[i] = map(float, next(self.datareader))
    return vs, ts

In [0]:
csv_reader = CSVReader(train_csv_path, batch_size)

In [72]:
vs, ts = next(csv_reader)
(vs, ts)

(array([12.,  6.,  8.,  5.,  8.,  8.,  9.,  7., -5.,  3.,  5.,  2.,  2.,
         3., -1.,  5.,  6.,  4.,  3.,  5.,  4.,  2.,  6.,  7.,  7.,  8.,
        14.,  9.,  4.,  7.,  4.,  3.]),
 array([1.46909998, 1.46909998, 1.46909998, 1.46909998, 1.46909998,
        1.46909998, 1.46909998, 1.46909998, 1.46909997, 1.46909997,
        1.46909997, 1.46909997, 1.46909997, 1.46909997, 1.46909997,
        1.46909997, 1.46909997, 1.46909996, 1.46909996, 1.46909996,
        1.46909996, 1.46909996, 1.46909996, 1.46909996, 1.46909996,
        1.46909996, 1.46909995, 1.46909995, 1.46909995, 1.46909995,
        1.46909995, 1.46909995]))